In [1]:
import pandas as pd
import glob
import tqdm
import numpy as np

In [2]:
path = r'C:\Users\Desktop\Desktop\rawData'

# All csv like files inside the pointed folder
files = glob.glob(path + '/*.csv')

data = []

for file in tqdm.tqdm(files):
    df = pd.read_csv(file, index_col=None, header=0)
    data.append(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:09<00:00,  1.83it/s]


In [3]:
complete = pd.concat(data, axis=0, ignore_index=True, sort=True)

In [4]:
complete.head(2)

,bike_id,bike_share_for_all_trip,duration_sec,end_station_id,end_station_latitude,end_station_longitude,end_station_name,end_time,member_birth_year,member_gender,start_station_id,start_station_latitude,start_station_longitude,start_station_name,start_time,user_type
0,96,NaN,80110,43.0,37.778768,-122.415929,San Francisco Public Library (Grove St at Hyde...,2018-01-01 15:12:50.2450,1987.0,Male,74.0,37.776435,-122.426244,Laguna St at Hayes St,2017-12-31 16:57:39.6540,Customer
1,88,NaN,78800,96.0,37.766210,-122.426614,Dolores St at 15th St,2018-01-01 13:49:55.6170,1965.0,Female,284.0,37.784872,-122.400876,Yerba Buena Center for the Arts (Howard St at ...,2017-12-31 15:56:34.8420,Customer


## Separating Station Data information
Since we will be working with many observations (over 2 million), it is a good idea to separate the station data information (id and name) in a separate table. This approach is more efficient and will improve the workflow performance.

In [5]:
endStationData = complete[['end_station_name', 'end_station_id']].copy()
startStationData = complete[['start_station_name','start_station_id']].copy()

In [6]:
endStationData.rename(columns={'end_station_name':'stationName', 'end_station_id':'stationId'}, inplace=True)
startStationData.rename(columns={'start_station_name':'stationName', 'start_station_id':'stationId'}, inplace=True);

In [7]:
stationData = pd.concat([endStationData, startStationData], sort=False, ignore_index=True)

In [8]:
stationData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508650 entries, 0 to 6508649
Data columns (total 2 columns):
stationName    object
stationId      float64
dtypes: float64(1), object(1)
memory usage: 99.3+ MB


In [9]:
stationData.drop_duplicates(inplace=True)

In [10]:
stationData.reset_index(drop=True)
stationData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 0 to 3026976
Data columns (total 2 columns):
stationName    394 non-null object
stationId      394 non-null float64
dtypes: float64(1), object(1)
memory usage: 9.3+ KB


Looks like there is 1 null entry. Let's investigate it:

In [11]:
null_entry = stationData[stationData.isnull().any(axis=1)]
null_entry

,stationName,stationId
1143064,NaN,NaN


Since it is just a null entry for both the station ID and station Name, let's go ahead and remove it from our station data table:

In [12]:
stationData.drop(index=null_entry.index, inplace=True)

In [13]:
stationData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 3026976
Data columns (total 2 columns):
stationName    394 non-null object
stationId      394 non-null float64
dtypes: float64(1), object(1)
memory usage: 9.2+ KB


In [14]:
# set stationId as string
stationData.stationId = stationData.stationId.astype(str);

In [15]:
stationData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 3026976
Data columns (total 2 columns):
stationName    394 non-null object
stationId      394 non-null object
dtypes: object(2)
memory usage: 9.2+ KB


Finally, we can save our Station Data as a csv file for future reference. Whenever necessary, we can merge the station name back to the station ID.

In [16]:
stationData.to_csv('stationData.csv', index=False)

## Preparing Complete Database
After separating the Station name data for future reference, when needed, let's prep the main dataset by dropping unecessary columns and investigating it's integrity:

In [17]:
complete.head(2)

,bike_id,bike_share_for_all_trip,duration_sec,end_station_id,end_station_latitude,end_station_longitude,end_station_name,end_time,member_birth_year,member_gender,start_station_id,start_station_latitude,start_station_longitude,start_station_name,start_time,user_type
0,96,NaN,80110,43.0,37.778768,-122.415929,San Francisco Public Library (Grove St at Hyde...,2018-01-01 15:12:50.2450,1987.0,Male,74.0,37.776435,-122.426244,Laguna St at Hayes St,2017-12-31 16:57:39.6540,Customer
1,88,NaN,78800,96.0,37.766210,-122.426614,Dolores St at 15th St,2018-01-01 13:49:55.6170,1965.0,Female,284.0,37.784872,-122.400876,Yerba Buena Center for the Arts (Howard St at ...,2017-12-31 15:56:34.8420,Customer


In [18]:
# Checking for missing data
complete.isnull().sum()

bike_id                         0
bike_share_for_all_trip    519700
duration_sec                    0
end_station_id              12501
end_station_latitude            0
end_station_longitude           0
end_station_name            12501
end_time                        0
member_birth_year          218166
member_gender              217733
start_station_id            12501
start_station_latitude          0
start_station_longitude         0
start_station_name          12501
start_time                      0
user_type                       0
dtype: int64

There are 12,501 observations without data for the Star/End station ID, which violates the assumption the bike was rented and returned.
Therefore, let's remove these entries:

In [19]:
missing_station_id = complete[complete.end_station_id.isnull()]
complete.drop(index=missing_station_id.index, inplace=True)

In [20]:
complete.isnull().sum()

bike_id                         0
bike_share_for_all_trip    519700
duration_sec                    0
end_station_id                  0
end_station_latitude            0
end_station_longitude           0
end_station_name                0
end_time                        0
member_birth_year          217832
member_gender              217399
start_station_id                0
start_station_latitude          0
start_station_longitude         0
start_station_name              0
start_time                      0
user_type                       0
dtype: int64

The "Bike share for All" is only available for data after 2017. It is a program providing special pricing for qualifying low-income users in the San Francisco Bay Area.
Let's fill all 2017 missing data as 'No', since the program didn't existed at the time:

In [21]:
complete.bike_share_for_all_trip.fillna('No', inplace=True);

In [22]:
# Transforming into binary 1:yes | 0:no
complete['low_income_trip'] = np.where(complete.bike_share_for_all_trip == 'Yes', 1,0)

In [23]:
# Droping bike-share-for-all-trip column
complete.drop(columns='bike_share_for_all_trip', inplace=True)

In [24]:
complete.member_birth_year.isnull().sum() / len(complete)

0.067194270879603585

We still have around 220 thousand observations missing data for birth year and gender. We could resort to imputation techniques, filling the missing values with the median and mode, for numerical and categorical variables, respectively.

But since our dataset has over 3.2 million observations, we can just drop these rows with missing values (6.7%) without compromising the representativeness of the data.

In [25]:
# Remove all entries with missing value
complete.dropna(inplace=True)

In [26]:
complete.isnull().sum()

bike_id                    0
duration_sec               0
end_station_id             0
end_station_latitude       0
end_station_longitude      0
end_station_name           0
end_time                   0
member_birth_year          0
member_gender              0
start_station_id           0
start_station_latitude     0
start_station_longitude    0
start_station_name         0
start_time                 0
user_type                  0
low_income_trip            0
dtype: int64

In [27]:
# Removing Station Name
complete.drop(columns=['start_station_name', 'end_station_name'], inplace=True)

In [28]:
# Transforming User Type into Subscriber (1:yes | 0:no)
complete.user_type.value_counts()

Subscriber    2681480
Customer       342512
Name: user_type, dtype: int64

In [29]:
complete['subscriber'] = np.where(complete.user_type == 'Subscriber',1,0)

In [30]:
complete.drop(columns='user_type', inplace=True)

In [31]:
# Transforming Gender into binary (1:male | 0:female)
complete['gender'] = np.where(complete.member_gender == 'Male', 1, 0)

In [32]:
complete.drop(columns='member_gender', inplace=True)

In [33]:
# After handling missing data, set appropriate data types
complete.end_station_id = complete.end_station_id.astype(int).astype(str)
complete.start_station_id = complete.start_station_id.astype(int).astype(str)
complete.member_birth_year = complete.member_birth_year.astype(int)

complete.end_time = pd.to_datetime(complete.end_time)
complete.start_time = pd.to_datetime(complete.start_time)

In [34]:
# Transforming duration from Seconds to Minutes
complete['duration_min'] = round(complete.duration_sec / 60, 1)

In [35]:
# Drop duration in seconds
complete.drop(columns='duration_sec', inplace=True)

In [36]:
# Calculate user age at time of usage
complete['user_age'] = complete.end_time.dt.year - complete.member_birth_year

In [37]:
# Drop member birth year
complete.drop(columns='member_birth_year', inplace=True)

In [38]:
# Extract time components from start time
complete['year'] = complete.start_time.dt.year
complete['month'] = complete.start_time.dt.month
complete['week'] = complete.start_time.dt.week
complete['day'] = complete.start_time.dt.day
complete['weekday'] = complete.start_time.dt.weekday # 0: Mon | 6: Sun
complete['hour'] = complete.start_time.dt.hour

In [39]:
# Drop End Time, since we already have start time and duration
complete.drop(columns='end_time', inplace=True)

In [40]:
complete.reset_index(drop=True).head(3)

,bike_id,end_station_id,end_station_latitude,end_station_longitude,start_station_id,start_station_latitude,start_station_longitude,start_time,low_income_trip,subscriber,gender,duration_min,user_age,year,month,week,day,weekday,hour
0,96,43,37.778768,-122.415929,74,37.776435,-122.426244,2017-12-31 16:57:39.654,0,0,1,1335.2,31,2017,12,52,31,6,16
1,88,96,37.766210,-122.426614,284,37.784872,-122.400876,2017-12-31 15:56:34.842,0,0,0,1313.3,53,2017,12,52,31,6,15
2,3167,247,37.867789,-122.265896,239,37.868813,-122.258764,2017-12-31 14:23:14.001,0,1,0,726.7,21,2017,12,52,31,6,14


In [41]:
complete.shape

(3023992, 19)

Finally, we are ready to save the final complete data into a csv file for further EDA.

In [42]:
complete.to_csv('Clean_FordGoBike_2017_2019.csv', index=False)